<a href="https://colab.research.google.com/github/samis922/corsera-html/blob/master/q_and_a_with_instructions_and_use_hint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [2]:
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
import torch

In [3]:
from google.colab import files
import pandas as pd

# Upload the file that contaion user instruction in CSV format
uploaded = files.upload()

Saving AP Test Scripts with User Instructions.csv to AP Test Scripts with User Instructions (1).csv


In [4]:
df = pd.read_csv("AP Test Scripts with User Instructions.csv")
print(df.head())

        Business Track Functional Track Test Script ID  \
0  Procure to Pay(P2P)         Payables         P2P-11   
1  Procure to Pay(P2P)         Payables         P2P-12   
2  Procure to Pay(P2P)         Payables         P2P-13   
3  Procure to Pay(P2P)         Payables         P2P-14   
4  Procure to Pay(P2P)         Payables         P2P-15   

                                    Test Script Name  \
0  Manage Transactions – Receive Invoices thru Sc...   
1  Manage Transactions -  Create invoice from a S...   
2               Manage Transactions - Create Invoice   
3          Manage Transactions - Create Credit Memos   
4    Manage Transactions - Create Prepayment Invoice   

                                   User Instructions  \
0  1.Test invoice creation using a scanned image\...   
1  1.Test invoice creation using data from a spre...   
2  1.Test standard invoice creation\n2. Test crea...   
3  1.Test Credit Memo creation for a Supplier/Ven...   
4  1.Test prepayment to a Supplier

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_checkpoint = "consciousAI/question-generation-auto-hints-t5-v1-base-s-q-c"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [6]:
print("Column names:", df.columns)


Column names: Index(['Business Track', 'Functional Track', 'Test Script ID',
       'Test Script Name', 'User Instructions', 'Description', 'Reqd.',
       'Review for SMHI', 'Smoke Test (Pass/Fail)',
       'Business Testing -Round 1 (Pass/Fail)',
       'End-2-End Testing (Pass/Fail)', 'Go-live', 'Comments',
       'Smoke Test Reference Page'],
      dtype='object')


In [7]:
question_list = []

for index, row in df.iterrows():

    context = f"question_context: {row['Description']} {row.get('User_Instructions', '')} hints: {'test selection'}"
    encodings = tokenizer.encode(context, return_tensors='pt', truncation=True, padding='max_length')
    output = model.generate(encodings, max_length=164,
                        min_length=20,
                        length_penalty=0.1,
                        num_return_sequences=3,
                        num_beams=4,
                        no_repeat_ngram_size=3,
                        early_stopping=True,
                        do_sample=True,
                        top_k =3,
                        #penalty_alpha =0,
                        #top_p=0,
                        temperature=1.1
                            )
    questions = [tokenizer.decode(id, clean_up_tokenization_spaces=False, skip_special_tokens=False) for id in output]
    question_list.append(questions)


KeyboardInterrupt: ignored

In [ ]:
df['Generated_Questions'] = question_list
df.to_csv('AP_Test_Scripts_with_Generated_Questions.csv', index=False)


In [ ]:
df['Generated_Questions'].head()

In [ ]:
files.download('AP_Test_Scripts_with_Generated_Questions.csv')